In [54]:
] activate ..

  Activating project at `~/Projects/UntitledLanguageProject`


In [55]:
using UntitledLanguage
import UntitledLanguage: create_lambda, val

In [56]:
tests = [
    """
    __begin__
        let fib(n)
           a: 1
           b: 0
           for _ in 0..n
              a, b: b, a+b
           return b
    """, 
    """
    set! foo (lambda (x) (+ x 3))
    """
]

2-element Vector{String}:
 "__begin__\n    let fib(n)\n      " ⋯ 47 bytes ⋯ "  a, b: b, a+b\n       return b\n"
 "set! foo (lambda (x) (+ x 3))\n"

In [57]:
function to_SExpr(t::Tuple)
    return SExpr(to_SExpr.(t))
end
function to_SExpr(s::Union{String, SubString})
    t = tryparse(Int64, s)
    if !(t isa Nothing)
        return Atom(t)
    end
    
    t = tryparse(Float64, s)
    if !(t isa Nothing)
        return Atom(t)
    end
    
    return Atom(Symbol(s))
end

to_SExpr (generic function with 2 methods)

In [58]:
function create_lambda(ast, env)
    body = ast.expr[3]
    if ast.expr[2] isa Atom
        name = val(ast.expr[2])
        return (args...) -> ast_walk(
            ast.expr[3],
            env = Environment(Dict{Symbol, Any}(name => args), env)
        )
    else
        names = val.(ast.expr[2].expr)
        return (args...) -> ast_walk(
            ast.expr[3], 
            env = Environment(env, names, args)
        )
    end
end

create_lambda (generic function with 1 method)

In [59]:
function split_comma(arr) # Not used, but for the future, for now only one item
    ret = []
    cur = []
    for a in arr
        if a != Symbol(",")
            push!(cur, a)
        else
            push!(ret, cur)
            cur = []
        end
    end
    push!(ret, cur)
    return [SExpr(tuple(r...)) for r in ret]
end

function split_comma(arr)
    @assert isodd(length(arr))
    @assert all(==(Atom(Symbol(","))).(arr[2:2:end]))
    return arr[1:2:end]
end
    

split_comma (generic function with 1 method)

In [60]:
BUILTINS = () -> Dict{Symbol, Any}(
    :__println__ => println,
    :add => (+),
    :mul => (*),
    :+ => (+),
    :* => (*),
    :is => (===),
    :(=) => (==),
    :> => (>),
    :< => (<), 
    :≥ => (≥), 
    :≤ => (≤),
    :list => (args...) -> SExpr(args),
    :__nl__ => Macro((args...) -> SExpr(args)),
    :__call__ => Macro((args...) -> se(args[1], split_comma(args[2:end])...)),
    :__tuple__ => Macro((args...) -> (println(args); SExpr(args))),
    :lambda_ => Macro((args...) -> SExpr((at(:lambda), SExpr(args[1].expr[2:end]), args[2]))), 
    :print => Macro((args...) -> se(:__println__, args...))
)

#74 (generic function with 1 method)

In [61]:
ast_walk(to_SExpr(parser("""
    begin
        set! x 3
        set! y 4
        set! x mul(x,x)
        print add(x,y)
""")), env=Environment(BUILTINS()))

13


In [62]:
ast_walk(to_SExpr(parser("""
    begin
        set! f (lambda_ (x) add(x, 3))
        print f(3)
""")), env=Environment(BUILTINS()))

(Atom{Symbol}(:lambda_), SExpr((Atom{Symbol}(:__tuple__), Atom{Symbol}(:x))), SExpr((Atom{Symbol}(:__call__), Atom{Symbol}(:add), Atom{Symbol}(:x), Atom{Symbol}(Symbol(",")), Atom{Int64}(3))))
6


In [63]:
# ast_walk(
temp = to_SExpr(parser("""
    begin
        set! x 3
        set! y 4
        add(x,y)
        3
        """)).expr[end-2]#, env=Environment(BUILTINS()))

SExpr((Atom{Symbol}(:__nl__), Atom{Symbol}(:set!), Atom{Symbol}(:y), Atom{Int64}(4)))

In [64]:
BUILTINS()[:__nl__](temp.expr[2:end])

SExpr(((Atom{Symbol}(:set!), Atom{Symbol}(:y), Atom{Int64}(4)),))

In [65]:
function repl(env=Environment(BUILTINS()); prompt="logos> ")
    while true
        if prompt != ""
            print(prompt)
        end
        str = readline()
        if str == "exit"
            break
        end
        try
            ast_walk(to_SExpr(parser(str)), env=env)
        catch e
            println(e)
        end
    end
end

repl (generic function with 2 methods)

In [66]:
repl(prompt="")

stdin>  set! x 3
stdin>  print x


3


stdin>  exit
